In [ ]:
%pip install google-genai

In [10]:
from google import genai
from google.genai import types
import os

PROJECT_ID = "cloud-assist-463215"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "us-central1"  # @param {type: "string"}
generation_model = "imagen-4.0-generate-001"
BUCKET_NAME = "marketing_media_data_beauty" # Replace with your bucket name
NUM_IMAGES_TO_GENERATE = 200 # The number of images you want to create

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [11]:
import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

In [12]:
import random

def generate_random_prompt():
    """
    Generates a single, highly specific and realistic prompt for a luxury cosmetic product.
    It intelligently randomizes product type, color, and generates on-product text.
    """

    # --- Product Categories and their specific, appropriate attributes ---

    # Eye Makeup
    eye_makeup_types = ["mascara wand", "eyeliner pencil", "eyeshadow palette"]
    eye_makeup_colors = ["black", "deep brown", "navy blue", "charcoal gray"]

    # Face Makeup
    face_makeup_types = ["foundation bottle", "blush compact", "concealer stick", "face powder compact"]
    face_makeup_colors = ["ivory", "beige", "sand", "tan", "porcelain", "cinnamon"]

    # Lip Makeup
    lip_makeup_types = ["lipstick tube", "lip gloss tube"]
    lip_makeup_colors = ["vibrant red", "soft pink", "nude", "coral", "burgundy", "peach"]

    # Skincare
    skincare_types = ["face cleanser", "face moisturizer", "sunscreen lotion", "serum dropper bottle", "hand cream tube"]
    skincare_descriptors = ["hydrating", "anti-aging", "purifying", "soothing", "firming"]

    # Haircare
    haircare_types = ["shampoo bottle", "conditioner bottle", "hair oil bottle", "hair spray can"]
    haircare_descriptors = ["volumizing", "smoothing", "nourishing", "repairing", "color-protecting"]

    # --- Choose a top-level category randomly ---
    category = random.choice(['eye_makeup', 'face_makeup', 'lip_makeup', 'skincare', 'haircare'])

    # --- Select attributes based on the chosen category ---
    if category == 'eye_makeup':
        product_type = random.choice(eye_makeup_types)
        color = random.choice(eye_makeup_colors)
        description = f"{color} {product_type}"
        product_label_text = f"{product_type.title().split()[0]}-{color.title()}"

    elif category == 'face_makeup':
        product_type = random.choice(face_makeup_types)
        color = random.choice(face_makeup_colors)
        description = f"{color} {product_type}"
        product_label_text = f"{product_type.title().split()[0]}-{color.title()}"

    elif category == 'lip_makeup':
        product_type = random.choice(lip_makeup_types)
        color = random.choice(lip_makeup_colors)
        description = f"{color} {product_type}"
        product_label_text = f"{product_type.title().split()[0]}-{color.title()}"

    elif category == 'skincare':
        product_type = random.choice(skincare_types)
        descriptor = random.choice(skincare_descriptors)
        description = f"{descriptor} {product_type}"
        product_label_text = f"{descriptor.title().split()[0]} {product_type.title().split()[0]}"

    else: # haircare
        product_type = random.choice(haircare_types)
        descriptor = random.choice(haircare_descriptors)
        description = f"{descriptor} {product_type}"
        product_label_text = f"{descriptor.title().split()[0]} {product_type.title().split()[0]}"

    # --- Base prompt template ---
    base_prompt_template = "A professional studio shot of a single, iconic luxury cosmetic product: a {description}. The product is perfectly centered and fully visible in a front-on, static view. The background is a pure white, seamless backdrop. The packaging has **clean, minimalist branding with text that says '{product_text}'** in a sans-serif font. The image is highly detailed and in high-resolution."

    # --- Format and return the final prompt ---
    return base_prompt_template.format(
        description=description,
        product_text=product_label_text
    )
prompt_array = []
# --- Example usage to generate 5 prompts ---
if __name__ == "__main__":
    print("Example prompts with intelligently generated, realistic text:")
    for i in range(200):
        prompt_array.append(generate_random_prompt())
print(f" {prompt_array[10]}")

Example prompts with intelligently generated, realistic text:
 A professional studio shot of a single, iconic luxury cosmetic product: a tan face powder compact. The product is perfectly centered and fully visible in a front-on, static view. The background is a pure white, seamless backdrop. The packaging has **clean, minimalist branding with text that says 'Face-Tan'** in a sans-serif font. The image is highly detailed and in high-resolution.


In [15]:
import io
import os
import random
import time
from datetime import datetime

from google.cloud import storage

import typing

import IPython.display
from PIL import Image as Image
from PIL import ImageOps as PIL_ImageOps

# --- Functions ---

def generate_and_upload_images(project_id: str, location: str, bucket_name: str, prompt: str, num_images: int):
    """
    Generates images using a Vertex AI foundation model and uploads them to a GCS bucket.
    """

    # Initialize the Google Cloud Storage client
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    print(f"Generating {num_images} images with prompt: '{prompt}'")

    for i in range(num_images):
            # Generate a single image

            image = client.models.generate_images(
                model=generation_model,
                prompt=prompt_array[i],
                config=types.GenerateImagesConfig(
                  aspect_ratio="16:9",
                  number_of_images=1,
                  image_size="2K",
                  safety_filter_level="BLOCK_MEDIUM_AND_ABOVE",
                  person_generation="ALLOW_ADULT",
                ),
            )

            #display_image(image.generated_images[0].image)


            # Get the raw image data as bytes
            image_bytes = image.generated_images[0].image.image_bytes

            # Use PIL to correctly save the raw bytes as a PNG file in memory
            # This is the crucial step that formats the image properly.
            img_byte_stream = io.BytesIO()
            pil_image = Image.open(io.BytesIO(image_bytes))
            pil_image.save(img_byte_stream, format="PNG")
            img_byte_stream.seek(0) # Rewind the buffer

            # Create a unique filename and upload the image stream
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            random_suffix = random.randint(1000, 9999)
            filename = f"beauty-product-{timestamp}-{random_suffix}.png"

            blob = bucket.blob(filename)
            blob.upload_from_file(img_byte_stream, content_type="image/png")

            print(f"✅ Successfully uploaded {filename} from prompt {prompt_array[i]}. ")

            # Add a small delay to prevent rate-limiting issues
            time.sleep(2)

    print("Image generation and upload process complete.")

# --- Run the script ---
if __name__ == "__main__":
    import vertexai

    # Call the function with your configuration
    generate_and_upload_images(
        project_id=PROJECT_ID,
        location=LOCATION,
        bucket_name=BUCKET_NAME,
        prompt="beauty things",
        num_images=NUM_IMAGES_TO_GENERATE
    )


Generating 200 images with prompt: 'beauty things'
✅ Successfully uploaded beauty-product-20250828_124125-7822.png from prompt A professional studio shot of a single, iconic luxury cosmetic product: a volumizing hair oil bottle. The product is perfectly centered and fully visible in a front-on, static view. The background is a pure white, seamless backdrop. The packaging has **clean, minimalist branding with text that says 'Volumizing Hair'** in a sans-serif font. The image is highly detailed and in high-resolution.. 
✅ Successfully uploaded beauty-product-20250828_124348-4190.png from prompt A professional studio shot of a single, iconic luxury cosmetic product: a anti-aging sunscreen lotion. The product is perfectly centered and fully visible in a front-on, static view. The background is a pure white, seamless backdrop. The packaging has **clean, minimalist branding with text that says 'Anti-Aging Sunscreen'** in a sans-serif font. The image is highly detailed and in high-resolution.

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7ba21d88b740>